In [1]:
#Essentials
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv 
import datetime as dt

#For scraping Reddit
import praw
from psaw import PushshiftAPI

#For EDA
import re
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.cluster import KMeans
import sklearn.cluster
import sklearn.decomposition
import matplotlib.colors
from textblob import TextBlob
from gensim.models import Word2Vec
import nltk
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import textstat
import gensim.corpora as corpora
from pprint import pprint
import gensim
from gensim.utils import simple_preprocess
import pyLDAvis.gensim_models
import pickle 
import pyLDAvis
import os

# Mental Heatlh

In [2]:
#Thanks to dmarx for the clear code on pulling from PSAW! PSAW docs are surprisingly unclear.
#https://github.com/dmarx/psaw
api = PushshiftAPI()

#Start date we're beginning to pull posts from
start_epoch=int(dt.datetime(2010, 1, 1).timestamp())


#Pulling submissions from mentalhealth (both the title and body of a post)

gen_subMH = list(api.search_submissions(after=start_epoch,
                            subreddit='mentalhealth',
                            filter=['title', 'id','score','link_flair_text','selftext'],
                            limit=1000000))


#Converting to dataframe
MHsub = pd.DataFrame([thing.d_ for thing in gen_subMH])


C:\Users\Julia\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
C:\Users\Julia\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\Julia\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


In [3]:
MHsub.to_csv('../Data/MHsub.csv',index=False)

In [4]:
#Pulling comments from mental health

gen_comMH = api.search_comments(after=start_epoch, 
                            subreddit='mentalhealth',
                            filter=['body', 'link_id', 'id', 'score','submission'],
                            #link_id = new_idsMH,
                            limit=35000000)

MHcom = pd.DataFrame([thing.d_ for thing in gen_comMH])

C:\Users\Julia\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
C:\Users\Julia\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\Julia\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


In [5]:
MHcom.to_csv('../Data/MHcom.csv',index=False)

# Mental Illness subreddit

In [6]:
gen_subMI = list(api.search_submissions(after=start_epoch,
                            subreddit='mentalillness',
                            filter=['title', 'id','score','link_flair_text','selftext'],
                            limit=1000000))


#Converting to dataframe
MIsub = pd.DataFrame([thing.d_ for thing in gen_subMI])

C:\Users\Julia\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
C:\Users\Julia\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\Julia\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


In [7]:
MIsub.to_csv('../Data/MIsub.csv',index=False)

In [8]:
gen_comMI = api.search_comments(after=start_epoch, 
                            subreddit='mentalillness',
                            filter=['body', 'link_id', 'id', 'score','submission'],
                            #link_id = new_MIids,
                            limit=35000000)

MIcom = pd.DataFrame([thing.d_ for thing in gen_comMI])

C:\Users\Julia\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
C:\Users\Julia\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
C:\Users\Julia\anaconda3\lib\site-packages\psaw\PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


In [9]:
MIcom.to_csv('../Data/MIcom.csv',index=False)

In [3]:
MHsub = pd.read_csv('../Data/MHsub.csv')
MHcom = pd.read_csv('../Data/MHcom.csv')
MIsub = pd.read_csv('../Data/MIsub.csv')
MIcom = pd.read_csv('../Data/MIcom.csv')

In [4]:
MHsub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364797 entries, 0 to 364796
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   created_utc      364797 non-null  int64  
 1   id               364797 non-null  object 
 2   score            364797 non-null  int64  
 3   selftext         343794 non-null  object 
 4   title            364797 non-null  object 
 5   created          364797 non-null  float64
 6   link_flair_text  141878 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 19.5+ MB


In [8]:
MHsub

,created_utc,id,score,selftext,title,created,link_flair_text
0,1663534517,xhrtbf,1,I don’t entirely want to reach out to people h...,How do I stop villainizing myself.,1.663549e+09,NaN
1,1663534244,xhrpbh,1,"I suffer from anxiety attacks regularly, usual...",is this a panic attack or something else?,1.663549e+09,NaN
2,1663533992,xhrlh6,1,I've been having these thoughts lately like\n\...,do you ever feel like not being alive anymore,1.663548e+09,NaN
3,1663533274,xhrats,1,"For some miraculous reason, I got a girlfriend...",Being normal,1.663548e+09,Opinion / Thoughts
4,1663533255,xhrajv,1,I've been going through a hard time lately and...,why do depressed people shave their heads,1.663548e+09,NaN
...,...,...,...,...,...,...,...
364792,1276360991,cease,1,NaN,Bipolar Blogger,1.276375e+09,NaN
364793,1276360127,ceang,0,NaN,Bipolar in Real Life,1.276375e+09,NaN
364794,1273693329,c3963,1,NaN,Texas Receives 'D' for Mental Health and Addic...,1.273708e+09,NaN
364795,1273002455,c02w6,1,NaN,Mental Health Threat: Growing Concern in Today...,1.273017e+09,NaN


In [9]:
MHcom

,body,created_utc,id,link_id,score,created
0,[removed],1663536087,iozbxkr,t3_xhsag5,1,1.663550e+09
1,Bro it seems you may actually delusional no ju...,1663536013,iozbqlk,t3_xhsag5,1,1.663550e+09
2,Thank you for for sharing. A reminder: if you ...,1663535972,iozbmub,t3_xhsf3f,1,1.663550e+09
3,Thanks,1663535958,iozbljf,t3_xhkzmw,1,1.663550e+09
4,I definitely have body image issues and I’m th...,1663535919,iozbi1l,t3_xhkzmw,1,1.663550e+09
...,...,...,...,...,...,...
1360964,Hello. I'll look someone up on the tubes out h...,1323379484,c36d25d,t3_n4rfh,1,1.323394e+09
1360965,"Hello, I work as a case manager at Momentum f...",1323377977,c36crmd,t3_n4rfh,1,1.323392e+09
1360966,"Hi,\n\nSee a doctor. A good one.\n\nI've suffe...",1322402458,c3340lm,t3_mdhe2,2,1.322417e+09
1360967,"you're not alone, i feel paranoia everywhere i...",1322024593,c322sel,t3_mdhe2,2,1.322039e+09


In [7]:
MHsub.link_flair_text.value_counts()

Question                           42488
Need Support                       35985
Venting                            31588
Opinion / Thoughts                 13673
Sadness / Grief                     5798
                                   ...  
*Serious Post, No memes please*        1
how are you?                           1
Venting Question                       1
Trauma                                 1
[Venting/Support]                      1
Name: link_flair_text, Length: 2021, dtype: int64

In [5]:
MIsub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50493 entries, 0 to 50492
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   created_utc      50493 non-null  int64  
 1   id               50493 non-null  object 
 2   link_flair_text  28390 non-null  object 
 3   score            50493 non-null  int64  
 4   selftext         41977 non-null  object 
 5   title            50493 non-null  object 
 6   created          50493 non-null  float64
dtypes: float64(1), int64(2), object(4)
memory usage: 2.7+ MB


In [10]:
#Get rid of emojis, get rid of removed comments